### Knock51: Data 加工の下準備をしよう
- Data 加工の下準備に入る前に、どのような機械学習 Model を構築するかを考える。
- 今回は、教師あり学習（二値分類）Model
    - 前月までの Data から、来月の Order 数が増加するのか減少するのか予測する Case は、Order 数自体を予測するのではなく、増加するのか減少するのか減少かを予測する。

In [1]:
# Directory 作成
import os

data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '1_output')
master_dir = os.path.join(data_dir, '99_master')
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

Data を読み込む際に File 複数が存在する場合、一つ一つ指定していたら大変なので、自動的に Directory の中身を取りに行き、配列として Path を定義する

In [2]:
# 注文 Data path の取得
import glob
tbl_order_file = os.path.join(input_dir, 'tbl_order_*.csv')
tbl_order_path = glob.glob(tbl_order_file)
tbl_order_path

['data\\0_input\\tbl_order_201904.csv',
 'data\\0_input\\tbl_order_201905.csv',
 'data\\0_input\\tbl_order_201906.csv',
 'data\\0_input\\tbl_order_201907.csv',
 'data\\0_input\\tbl_order_201908.csv',
 'data\\0_input\\tbl_order_201909.csv',
 'data\\0_input\\tbl_order_201910.csv',
 'data\\0_input\\tbl_order_201911.csv',
 'data\\0_input\\tbl_order_201912.csv',
 'data\\0_input\\tbl_order_202001.csv',
 'data\\0_input\\tbl_order_202002.csv',
 'data\\0_input\\tbl_order_202003.csv']

### Knock52: Data の読み込みを行ない加工の方向性を検討しよう

In [3]:
# Master data の読み込み
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


Directory 構造をしっかり考え、変数名を統一することで、簡単に使い回すことができる。

In [4]:
# 注文 Data の読み込み
tbl_order_path = tbl_order_path[0]
print(f"読み込み Data: {tbl_order_path}")
order_data = pd.read_csv(tbl_order_path)
print(f"Data 件数: {len(order_data)}")
order_data.head(3)

読み込み Data: data\0_input\tbl_order_201904.csv
Data 件数: 233393


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,22808272,39,C80973292,40,22222408,2019-04-01 11:00:00,2019-04-01 11:26:00,1,2112,1
1,10902625,63,C94948343,20,79467084,2019-04-01 11:00:00,2019-04-01 11:47:00,0,2154,2
2,5990375,63,C91814442,1,61749935,2019-04-01 11:00:00,2019-04-01 11:10:00,0,3050,2


Data 件数が非常に多い（7桁くらい）場合、Data をやり取りするには、処理に時間がかかってしまう。
よって、まずは少量の Data 量で加工の方向を考え、全期間の Data に適用する。